In [ ]:
import rasterio
from rasterio.windows import from_bounds
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
def read_window(cog_path, bbox):
    with rasterio.open(cog_path) as src:
        window = from_bounds(*bbox, transform=src.transform)
        data = src.read(1, window=window, masked=True)
    return data

In [ ]:
def mean_value_calculation(cogs, bbox):
    arrays = []
    for cog in cogs:
        try:
            arr = read_window(cog, bbox)
            arrays.append(arr)
        except Exception as e:
            print(f"Error processing {cog}: {e}")
    if not arrays:
        print("No valid COGs processed.")
        return
    # Stack and calculate pixel-wise mean
    stack = np.ma.stack(arrays)
    mean_img = np.ma.mean(stack, axis=0)
    # Display
    plt.figure(figsize=(8, 6))
    plt.imshow(mean_img, cmap="viridis")
    plt.colorbar(label="Mean Value")
    plt.title("Pixel-wise Mean Across COGs")
    plt.xlabel("Column")
    plt.ylabel("Row")
    plt.show()

cogs = "https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2025/07/02/S2C_20250702_latn537lonw0037_T30UVE_ORB037_20250702133100_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.tif,https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_1/2025/06/30/S1A_20250630_125_desc_063918_063943_VVVH_G0_NI_TM65_RTCK_SpkRL.tif,https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_1/2025/06/28/S1A_20250628_103_asc_180657_180722_VVVH_G0_GB_OSGB_RTCK_SpkRL.tif"

In [ ]:
cogs = cogs.split(",")

['https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2025/07/02/S2C_20250702_latn537lonw0037_T30UVE_ORB037_20250702133100_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.tif',
 'https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_1/2025/06/30/S1A_20250630_125_desc_063918_063943_VVVH_G0_NI_TM65_RTCK_SpkRL.tif',
 'https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_1/2025/06/28/S1A_20250628_103_asc_180657_180722_VVVH_G0_GB_OSGB_RTCK_SpkRL.tif']

In [ ]:
mean_value_calculation(cogs, bbox)